In [6]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
from requests.exceptions import ConnectionError
# from pywebcopy import save_webpage

In [7]:

internal_urls = set()
external_urls = set()
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls

total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except:
        print("caught")


In [8]:
base_urls = ["https://www.okwu.edu/coronavirus/",
"https://harrisburg.psu.edu/feature/coronavirus-information-penn-state-harrisburg",
"https://www.pointpark.edu/StudentLife/HealthandStudentServices/StudentHealthCenter/returnToCampus/index",
"https://www.pnw.edu/pnw-and-coronavirus-covid-19/",
"https://www.ramapo.edu/health/coronavirus-covid-19/",
"https://www.francis.edu/coronavirus/",
"https://www.stmarys-ca.edu/public-safety-transportation/coronavirus-covid-information-resources",
"https://www.sjsu.edu/healthadvisories/",
"https://sienaheights.edu/Siena-Heights-News/ArtMID/2940/ArticleID/687/FAQs-About-SHU-and-Coronavirus-CARES-Act-FAQ"]

In [9]:
for base_url in base_urls:
    internal_urls.add(base_url)
    parsedurl = urlparse(base_url)
    print(parsedurl)
    base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
    print(base_url_main)
    # base_url_text = base_url.split("//",1)[1]
    base_url_text = parsedurl.netloc+parsedurl.path
    print(base_url_text)
    # base_url_text_domain = base_url_text.split("/",1)[0]
    base_url_text_domain = parsedurl.netloc
    print(base_url_text_domain)
    crawl(base_url)
    print("[+] Total External links:", len(external_urls))
    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total:", len(external_urls) + len(internal_urls))
    count = 0
    last_count = 989874
    immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

    for i in range(0,4):
        for url in internal_urls:
            if base_url_text in url:
                immediate_urls.add(url)
        count = len(immediate_urls)
        print(count,i)
        if(count == last_count):
            break
        last_count = count
        for immediate_url in immediate_urls:
            try:
                crawl(immediate_url)
                print("[+] NEW Total External links:", len(external_urls))
                print("[+] NEW Total Internal links:", len(internal_urls))
                print("[+] NEW Total:", len(external_urls) + len(internal_urls))
                if len(internal_urls)>1000:
                    break
            except requests.exceptions.ConnectionError:
                continue

    def removeAfterN(yourStr, nth, occurenceOf):
        return occurenceOf.join(yourStr.split(occurenceOf)[:nth])
    
    
    semifinal_immediate_urls = set()

    for immediate_url in immediate_urls:
        stripped_url = removeAfterN(immediate_url,2,"http")


        semifinal_immediate_urls.add(stripped_url)
    #     final_immediate_urls.add(rest)
    #     print(rest)



    for i in semifinal_immediate_urls:
        print(i)
    total_count = len(semifinal_immediate_urls)
    print(total_count)
    
    
    final_immediate_urls = set()
    for semifinal_immediate_url in semifinal_immediate_urls:
        if "tel:" not in semifinal_immediate_url:
            final_immediate_urls.add(semifinal_immediate_url)


    for i in final_immediate_urls:
        print(i)
    total_count = len(final_immediate_urls)
    print(total_count)       
    
    
    
    parent_folder = "Masters" #Change based on folder
    name_of_folder = f'{base_url_text_domain}_Screenshots'
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
    if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
        os.makedirs(f'{parent_folder}\\{name_of_folder}')
    
    
    
    webpage_save = set()
    webpage_save_count = 0
    webpage_cannot_save = 0
    for i in final_immediate_urls:
        if ' ' in i:
            continue
        try:
            code=urlopen(i).getcode()
            webpage_save.add(i)
            webpage_save_count = webpage_save_count+1
            print(f"{code} can_save - {i}")
        except (urllib.error.HTTPError,urllib.error.URLError) as e:
            print("discarded")
            webpage_cannot_save = webpage_cannot_save+1
            continue

    print(f'Webpages save as HTML :{webpage_save_count}')
    print(f'Webpages unable to save as HTML :{webpage_cannot_save}')

    
    pdf_count = 0
    pdf_urls = set()
    for webpage in webpage_save:
        try:
            if webpage[-4:] == ".pdf":
                url = webpage
                file_name = url.replace(base_url,'')
                file_name = file_name.replace('/','')
                if file_name == '':
                    file_name = "Home"
                pdf_urls.add(webpage)
                print(webpage)
                r = requests.get(url, stream=True)
                with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                    for chunk in r.iter_content(2000):
                        fd.write(chunk)
                pdf_count = pdf_count + 1
        except:
            print("caught")

    print(f"Pdfs saved: {pdf_count}")
    
    
    
    final_immediate_urls = final_immediate_urls - pdf_urls
    webpage_save = webpage_save - pdf_urls
    web_save_count = len(webpage_save)
    total_count = len(final_immediate_urls)
    print(total_count) 
    
    
    
    final_webpage_save_count = webpage_save_count - pdf_count
    print(final_webpage_save_count)
    
    
    webpage_save_counter_loop = 0
    for webpage in webpage_save:
        try:
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
            webpage_save_counter_loop = webpage_save_counter_loop+1
            print(f'{webpage_save_counter_loop} of {final_webpage_save_count} - {webpage}')
        except:
            print("caught")

    print("Saving Completed")
    
    
    count = 0
    for i in final_immediate_urls:
        try:
            url = i
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            count = count + 1
            print(f"{count} of {total_count}")
            print(f'Visiting {base_url_text}/{file_name}')
            print(f'...Taking a screenshot')
            driver.get(url)
            S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
            driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
            driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
            print(f'Screenshot of {file_name} page taken! \n')
        except:
            print("caught")
    #     print("Saving HTML")
    #     pyautogui.hotkey('ctrl', 's')
    #     time.sleep(1)
    #     pyautogui.typewrite(f'{file_name}.html')
    #     pyautogui.hotkey('enter')
    driver.quit()

    print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')

ParseResult(scheme='https', netloc='www.okwu.edu', path='/coronavirus/', params='', query='', fragment='')
https://www.okwu.edu
www.okwu.edu/coronavirus/
www.okwu.edu
[+] Total External links: 0
[+] Total Internal links: 1
[+] Total: 1
1 0
[+] NEW Total External links: 0
[+] NEW Total Internal links: 1
[+] NEW Total: 1
1 1
https://www.okwu.edu/coronavirus/
1
https://www.okwu.edu/coronavirus/
1
discarded
Webpages save as HTML :0
Webpages unable to save as HTML :1
Pdfs saved: 0
1
0
Saving Completed
1 of 1
Visiting www.okwu.edu/coronavirus//Home
...Taking a screenshot
Screenshot of Home page taken! 

Task Completed! Files stored in the Masters\www.okwu.edu_Screenshots Folder
ParseResult(scheme='https', netloc='harrisburg.psu.edu', path='/feature/coronavirus-information-penn-state-harrisburg', params='', query='', fragment='')
https://harrisburg.psu.edu
harrisburg.psu.edu/feature/coronavirus-information-penn-state-harrisburg
harrisburg.psu.edu
[INT] Internal link: https://harrisburg.psu.ed

[INT] Internal link: https://harrisburg.psu.edu/marketing-research-and-communications
[INT] Internal link: https://harrisburg.psu.edu/file/10376
[INT] Internal link: https://harrisburg.psu.edu/marketing-research-and-communications/photo-request-form
[INT] Internal link: https://harrisburg.psu.edu/marketing-research-and-communications/publication-request-form
[INT] Internal link: https://harrisburg.psu.edu/marketing-research-and-communications/story-idea-request-form
[INT] Internal link: https://harrisburg.psu.edu/marketing-and-communications/materials-review-request-form
[INT] Internal link: https://harrisburg.psu.edu/marketing-research-and-communications/people-database-update
[INT] Internal link: https://harrisburg.psu.edu/marketing-research-and-communications/website-web-application-change-request-form
[INT] Internal link: https://harrisburg.psu.edu/campus-events-calendar-new-submission-process
[INT] Internal link: https://harrisburg.psu.edu/marketing-and-communications
[INT] Intern

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[INT] Internal link: https://harrisburg.psu.edu/faculty-and-staff/kalisha-devan
[INT] Internal link: https://harrisburg.psu.edu/faculty-and-staff/yvonne-harhigh
[INT] Internal link: https://harrisburg.psu.edu/faculty-and-staff/panna-idowu-ma
[INT] Internal link: https://harrisburg.psu.edu/faculty-and-staff/ilhan-kucukaydin-edd
[INT] Internal link: https://harrisburg.psu.edu/faculty-and-staff/christy-lerch
[INT] Internal link: https://harrisburg.psu.edu/faculty-and-staff/alixandra-moyer
[INT] Internal link: https://harrisburg.psu.edu/faculty-and-staff/daniel-poeschl
[INT] Internal link: https://harrisburg.psu.edu/faculty-and-staff/sharon-siegfried
[INT] Internal link: https://harrisburg.psu.edu/faculty-and-staff/simon-song
[INT] Internal link: https://harrisburg.psu.edu/science-engineering-technology/faculty-and-staff/christopher-weaver
[INT] Internal link: https://harrisburg.psu.edu/behavioral-sciences-and-education
[INT] Internal link: https://harrisburg.psu.edu/humanities
[INT] Inter

[+] NEW Total External links: 131
[+] NEW Total Internal links: 232
[+] NEW Total: 363
1 1
https://www.pointpark.edu/StudentLife/HealthandStudentServices/StudentHealthCenter/returnToCampus/index
1
https://www.pointpark.edu/StudentLife/HealthandStudentServices/StudentHealthCenter/returnToCampus/index
1
200 can_save - https://www.pointpark.edu/StudentLife/HealthandStudentServices/StudentHealthCenter/returnToCampus/index
Webpages save as HTML :1
Webpages unable to save as HTML :0
Pdfs saved: 0
1
1
1 of 1 - https://www.pointpark.edu/StudentLife/HealthandStudentServices/StudentHealthCenter/returnToCampus/index
Saving Completed
1 of 1
Visiting www.pointpark.edu/StudentLife/HealthandStudentServices/StudentHealthCenter/returnToCampus/index/Home
...Taking a screenshot
Screenshot of Home page taken! 

Task Completed! Files stored in the Masters\www.pointpark.edu_Screenshots Folder
ParseResult(scheme='https', netloc='www.pnw.edu', path='/pnw-and-coronavirus-covid-19/', params='', query='', fragme

...

[INT] Internal link: https://www.sjsu.edu/healthadvisories/campus-messages/
[INT] Internal link: https://www.sjsu.edu/police/prepare/alert_sjsu/index.html
[INT] Internal link: https://www.sjsu.edu/healthadvisories/campus-messages/022820-sjsu-responds-to-covid19/tel:4089241000
[+] NEW Total External links: 447
[+] NEW Total Internal links: 1174
[+] NEW Total: 1621
46 1
[+] NEW Total External links: 447
[+] NEW Total Internal links: 1174
[+] NEW Total: 1621
46 2
https://www.sjsu.edu/healthadvisories/campus-messages/022820-sjsu-responds-to-covid19/index.php
https://www.sjsu.edu/healthadvisories/campus-messages/012920-healthy-habits/index.php
https://www.sjsu.edu/healthadvisories/current-update/index.php
https://www.sjsu.edu/healthadvisories/faq/index.php
https://www.sjsu.edu/healthadvisories/campus-messages/032720-zoom-bombing/index.php
https://www.sjsu.edu/healthadvisories/report-a-case.php
https://www.sjsu.edu/healthadvisories/campus-messages/031220-majority-of-events-canceled/index.php

200 can_save - https://www.sjsu.edu/healthadvisories/report-a-case.php
200 can_save - https://www.sjsu.edu/healthadvisories/campus-messages/031220-majority-of-events-canceled/index.php
200 can_save - https://www.sjsu.edu/healthadvisories/campus-messages/051220-virtual-fall-2020/index.php
200 can_save - https://www.sjsu.edu/healthadvisories/campus-messages/040420-spring-2020-grading/index.php
200 can_save - https://www.sjsu.edu/healthadvisories/campus-messages/040120-shelter-in-place-extended/index.php
200 can_save - https://www.sjsu.edu/healthadvisories/campus-messages/031620-shelter-in-place/index.php
200 can_save - https://www.sjsu.edu/healthadvisories/campus-messages/032120-library-closed/index.php
200 can_save - https://www.sjsu.edu/healthadvisories/current-update/advisory-log/index.php
200 can_save - https://www.sjsu.edu/healthadvisories/campus-messages/030520-emergency-operations-center/index.php
200 can_save - https://www.sjsu.edu/healthadvisories/campus-messages/032720-second-c

Screenshot of campus-messages012920-healthy-habitsindex.php page taken! 

3 of 44
Visiting www.sjsu.edu/healthadvisories//current-updateindex.php
...Taking a screenshot
Screenshot of current-updateindex.php page taken! 

4 of 44
Visiting www.sjsu.edu/healthadvisories//faqindex.php
...Taking a screenshot
Screenshot of faqindex.php page taken! 

5 of 44
Visiting www.sjsu.edu/healthadvisories//campus-messages032720-zoom-bombingindex.php
...Taking a screenshot
Screenshot of campus-messages032720-zoom-bombingindex.php page taken! 

6 of 44
Visiting www.sjsu.edu/healthadvisories//report-a-case.php
...Taking a screenshot
Screenshot of report-a-case.php page taken! 

7 of 44
Visiting www.sjsu.edu/healthadvisories//campus-messages031220-majority-of-events-canceledindex.php
...Taking a screenshot
Screenshot of campus-messages031220-majority-of-events-canceledindex.php page taken! 

8 of 44
Visiting www.sjsu.edu/healthadvisories//campus-messages051220-virtual-fall-2020index.php
...Taking a screen

[INT] Internal link: https://sienaheights.edu/
[INT] Internal link: https://sienaheights.edu/Academics/MajorsPrograms.aspx
[INT] Internal link: http://giving.sienaheights.edu
[INT] Internal link: https://sienaheights.edu/Locations
[INT] Internal link: https://sienaheights.edu/Locations/Adrian-Main-Campus
[INT] Internal link: https://sienaheights.edu/Locations/Battle-Creek
[INT] Internal link: https://sienaheights.edu/Locations/Benton-Harbor
[INT] Internal link: https://sienaheights.edu/Locations/Dearborn
[INT] Internal link: https://sienaheights.edu/Locations/Diocese-of-Lansing
[INT] Internal link: https://sienaheights.edu/Locations/Jackson
[INT] Internal link: https://sienaheights.edu/Locations/Kalamazoo
[INT] Internal link: https://sienaheights.edu/Locations/Lansing
[INT] Internal link: https://sienaheights.edu/Locations/Metro-Detroit
[INT] Internal link: https://sienaheights.edu/Locations/Monroe
[INT] Internal link: https://sienaheights.edu/Locations/Online-Learning-Program
[INT] In

[+] NEW Total External links: 469
[+] NEW Total Internal links: 1312
[+] NEW Total: 1781
1 1
https://sienaheights.edu/Siena-Heights-News/ArtMID/2940/ArticleID/687/FAQs-About-SHU-and-Coronavirus-CARES-Act-FAQ
1
https://sienaheights.edu/Siena-Heights-News/ArtMID/2940/ArticleID/687/FAQs-About-SHU-and-Coronavirus-CARES-Act-FAQ
1
200 can_save - https://sienaheights.edu/Siena-Heights-News/ArtMID/2940/ArticleID/687/FAQs-About-SHU-and-Coronavirus-CARES-Act-FAQ
Webpages save as HTML :1
Webpages unable to save as HTML :0
Pdfs saved: 0
1
1
1 of 1 - https://sienaheights.edu/Siena-Heights-News/ArtMID/2940/ArticleID/687/FAQs-About-SHU-and-Coronavirus-CARES-Act-FAQ
Saving Completed
1 of 1
Visiting sienaheights.edu/Siena-Heights-News/ArtMID/2940/ArticleID/687/FAQs-About-SHU-and-Coronavirus-CARES-Act-FAQ/Home
...Taking a screenshot
Screenshot of Home page taken! 

Task Completed! Files stored in the Masters\sienaheights.edu_Screenshots Folder


In [10]:
print("DONE")

DONE
